In [1]:
# Some standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import scipy.io as scio
import torch.nn.functional as F
from torch.nn import Module, Parameter, init

In [2]:
class ModulatorNet(nn.Module):
    def __init__(self, signal_dimension, basis_function_length, samples_per_symbol):
        super(ModulatorNet, self).__init__()
        self._signal_dimension = signal_dimension
        self._basis_function_length = basis_function_length
        self._samples_per_symbol = samples_per_symbol
        self._combination_weight = torch.Tensor([[1,0,0,-1],[0,1,1,0]])

        # Signal mapper via transposed convolutional layer
        self.signal_part = nn.ConvTranspose1d(in_channels=2*self._signal_dimension, 
                                        out_channels=4, 
                                        groups=2, 
                                        kernel_size=self._basis_function_length, 
                                        stride=self._samples_per_symbol, 
                                        bias=False)
        # self.signal_comb = nn.Linear(in_features=4,out_features=2,bias=False)

    def forward(self, symbol_vetor):
        # Get components for real and imaginary parts
        signal_components = self.signal_part(symbol_vetor)
        # Transpose channel <-> length
        signal_components_T = torch.transpose(signal_components, dim0=1, dim1=2)
        # Combine components
        signal_tx = F.linear(input=signal_components_T,weight=self._combination_weight,bias=None)
        # signal_tx = self.signal_comb(signal_components_T)
        
        return signal_tx

In [3]:
# Load Data symbol
# Final symbol tensors will have a shape of (Batch, Channel, Length)
Symbol_file = scio.loadmat('./TrainingWaveform/QAM/QAMSymbol_batch.mat')
Symbol = Symbol_file['QAMSymbol_batch']
print(Symbol.shape)
# Symbol matrix has a shape of (Batch, Channel, Length)
signal_dimension = Symbol.shape[1]
print(signal_dimension)
# Extract real and imaginary parts to form the input mat
Symbol_real = np.real(Symbol)
Symbol_imag = np.imag(Symbol)
Symbol_mat = np.concatenate((Symbol_real, Symbol_imag), axis = 1).astype('float32')

# Add a dimension at 0 for Batch
Symbol_tensor = torch.tensor(Symbol_mat)
print(Symbol_tensor.shape)

# Load Waveform
# Final Waveform tensors will have a shape of (Batch, Length, 2)
Waveform_file = scio.loadmat('./TrainingWaveform/QAM/QAMSignal_batch.mat')
Waveform = Waveform_file['QAMSignal_batch']
print(Waveform.shape)
# Waveform matrix has a shape of (Batch, Length)
# Extract real and imaginary parts to form the input mat
Waveform_real = np.real(Waveform)
Waveform_imag = np.imag(Waveform)
Waveform_mat = np.stack((Waveform_real, Waveform_imag), axis = 2).astype('float32')

Waveform_tensor = torch.tensor(Waveform_mat)

print(Waveform_tensor.shape)

(128, 1, 64)
1
torch.Size([128, 2, 64])
(128, 285)
torch.Size([128, 285, 2])


In [4]:
# Configure basis function
basis_fucntion_file = scio.loadmat('./TrainingWaveform/QAM/rrc_filter_taps.mat')
basis_fucntions = basis_fucntion_file['rrc_filter_taps']
# Basis function matrix has a shape of (Dimensions, Length) 
basis_fucntion_length = basis_fucntions.shape[1]
# Extract real and imaginary parts of basis functions
basis_real_tensor = torch.Tensor(np.real(basis_fucntions)).unsqueeze(dim=1)
basis_imag_tensor = torch.Tensor(np.imag(basis_fucntions)).unsqueeze(dim=1)
basis_tensor = torch.concat([basis_real_tensor,basis_imag_tensor],dim=1)
basis_tensor = torch.concat([basis_tensor,basis_tensor],dim=0)
# Configure samples per symbol
samples_per_symbol = 4
print(basis_tensor.shape)

torch.Size([2, 2, 33])


/tmp/ipykernel_156684/3615671927.py:8: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/torch/csrc/utils/tensor_numpy.cpp:199.)
  basis_imag_tensor = torch.Tensor(np.imag(basis_fucntions)).unsqueeze(dim=1)


In [5]:
# Use ModulatorNet to generate signal
QAM_Modulator = ModulatorNet(signal_dimension=signal_dimension, 
                        basis_function_length=basis_fucntion_length,
                        samples_per_symbol=samples_per_symbol)
QAM_Modulator.signal_part.weight.data = basis_tensor
QAM_Waveform = QAM_Modulator(Symbol_tensor)
print(QAM_Waveform.shape)

torch.Size([128, 285, 2])


In [6]:
# Convert to ONNX
import torch.onnx
QAM_Modulator.eval()
InputSymbol = Symbol_tensor
OutputWaveform = QAM_Modulator(InputSymbol)
torch.onnx.export(QAM_Modulator,InputSymbol,"QAM_RRC_Modulator.onnx", export_params=True, 
                    opset_version=13,input_names = ['inputsymbol'],output_names = ['outputwaveform'],
                    dynamic_axes={'inputsymbol' : {0 : 'batch_size', 2: 'symbol_length'},    
                                'outputwaveform' : {0 : 'batch_size', 1: 'waveform_length'}})